<a href="https://colab.research.google.com/github/Davidkariuki008/self-trial/blob/master/Space_Invaders_Davis_Kariuki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install Dependencies

In [ ]:
pip install tensorflow gym keras-rl2 gym[atari]

# 1. Test Random Environment with OpenAI Gym

In [ ]:
import gym 
import random

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
mkdir: cannot create directory ‘rars’: File exists
copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Air Raid (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying crazy_climber.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Crazy Climber.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/crazy_climber.bin
copying 

In [ ]:
env = gym.make('SpaceInvaders-v1')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode='rgb_array')
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:310.0
Episode:2 Score:60.0
Episode:3 Score:110.0
Episode:4 Score:110.0
Episode:5 Score:155.0


# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    global model
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
#del model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 6)                

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
dqn.fit(env, nb_steps=1000, visualize=False, verbose=2)

Training for 1000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
 790/1000: episode: 1, duration: 31.596s, episode steps: 790, steps per second:  25, episode reward: 155.000, mean reward:  0.196 [ 0.000, 30.000], mean action: 2.504 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
done, took 39.135 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=50, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 50 episodes ...
Episode 1: reward: 75.000, steps: 643
Episode 2: reward: 125.000, steps: 684
Episode 3: reward: 120.000, steps: 602
Episode 4: reward: 155.000, steps: 875
Episode 5: reward: 120.000, steps: 707
Episode 6: reward: 75.000, steps: 657
Episode 7: reward: 75.000, steps: 647
Episode 8: reward: 230.000, steps: 837
Episode 9: reward: 110.000, steps: 678
Episode 10: reward: 315.000, steps: 1407
Episode 11: reward: 105.000, steps: 656
Episode 12: reward: 90.000, steps: 681
Episode 13: reward: 105.000, steps: 677
Episode 14: reward: 105.000, steps: 673
Episode 15: reward: 100.000, steps: 706
Episode 16: reward: 155.000, steps: 812
Episode 17: reward: 95.000, steps: 650
Episode 18: reward: 105.000, steps: 620
Episode 19: reward: 105.000, steps: 765
Episode 20: reward: 75.000, steps: 623
Episode 21: reward: 75.000, steps: 661
Episode 22: reward: 75.000, steps: 646
Episode 23: reward: 120.000, steps: 640
Episode 24: reward: 135.000, steps: 769
Episode 25: reward: 75.000, 

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [ ]:
#del model, dqn

In [ ]:
dqn.load_weights('SavedWeights/10k-Fast/dqn_weights.h5f')